In [2]:
import argparse
from SoccerNet.utils import getListGames
import os
import json
from tqdm import tqdm
import numpy as np
from SoccerNet.Evaluation.ActionSpotting import evaluate

In [3]:
parser = argparse.ArgumentParser()
# parser.add_argument("--anno_file",default='')
parser.add_argument("--source_test_path",default='/youtu_pedestrian_detection/zhuhe/soccernet/submit/size_224_with_bg')#输入test文件夹 
parser.add_argument("--output_test_path",default='/youtu_pedestrian_detection/zhuhe/soccernet/submit/output')  #筛选后test文件夹

parser.add_argument("--source_challenge_path",default='/youtu_pedestrian_detection/chengzhilin/program/soccernet/models/MVIT_WIN5/outputs_challenge')#输入challenge文件夹 
parser.add_argument("--output_challenge_path", default="/youtu_pedestrian_detection/chengzhilin/program/soccernet/process/challenge")  #输出challenge文件夹
parser.add_argument('--window_start', type=int,   default=10,     help='Size of the chunk (in seconds)' )
parser.add_argument('--window_end', type=int,   default=20,     help='Size of the chunk (in seconds)' )
parser.add_argument('--window_gap',  type=int,   default=0.5,     help='Size of the chunk (in seconds)' )

args=parser.parse_args([])

In [4]:
os.makedirs(args.output_challenge_path,exist_ok='True')

In [5]:
def process_window(window):
    action_dic={}
    res=[]
    for pre in window:
        if pre['label'] not in action_dic:
            action_dic[pre['label']]=pre
        else:
            if pre['confidence']>action_dic[pre['label']]['confidence']:
                action_dic[pre['label']]=pre
    for label in action_dic.keys():
        res.append(action_dic[label])
    return res

In [ ]:
total_results=[]
window_size=args.window_start
while window_size<=args.window_end:
    print(window_size)

    for cata in getListGames(split="test"):

        file=os.path.join(args.source_test_path,cata,'results_spotting.json')
        with open(file,'r') as f:
            data=json.load(f)
        pres=data['predictions']
        pres.sort(key=lambda x:(int(x['half']),int(x['position'])))

        json_data = {} 
        total_pre={}
        json_data["UrlLocal"]=data['UrlLocal']
        json_data["predictions"]=[]

        last_half=1
        start,end=0,window_size
        window=[]
        for cur_pre,pre in enumerate(pres):


            if start<= int(pre['position'])/1000.0 <end:
                window.append(pre)
            else:
                json_data["predictions"]+=process_window(window)
                window=[pre]
                # while end<int(pre['position']):
                start+=window_size
                end+=window_size

                if last_half==1 and int(pre['half'])==2:
                    last_half=2
                    start,end=0,window_size

        json_data["predictions"]+=process_window(window)

        os.makedirs(os.path.join(args.output_test_path,cata),exist_ok='True')
        with open(os.path.join(args.output_test_path,cata,'results_spotting.json'),'w') as f:
            json.dump(json_data,f,indent=2)
    
    PATH_DATASET='/youtu_pedestrian_detection/junweil/action_datasets/soccernet'
    PATH_PREDICTIONS=args.output_test_path


    from SoccerNet.Evaluation.ActionSpotting import evaluate
    results = evaluate(SoccerNet_path=PATH_DATASET, Predictions_path=PATH_PREDICTIONS,
                       split="test", version=2, prediction_file="results_spotting.json", metric="tight")

    a_map,a_mAP_visible,a_mAP_unshown=results["a_mAP"],results["a_mAP_visible"],results["a_mAP_unshown"]
    total_results.append([window_size,a_map,a_mAP_visible,a_mAP_unshown])
    
    
    window_size+=args.window_gap

10


100%|██████████| 5/5 [00:14<00:00,  2.90s/it]


10.5


100%|██████████| 5/5 [00:14<00:00,  2.83s/it]


11.0


...

15.0


100%|██████████| 5/5 [00:13<00:00,  2.68s/it]


15.5


100%|██████████| 5/5 [00:13<00:00,  2.70s/it]


16.0


In [111]:
total_results.sort(key=lambda x:-x[1])
window_size=total_results[0][0]

In [112]:
window_size

10.0

In [113]:
total_results

[[10.0, 0.3635886168702596, 0.4452361951226681, 0.13416065650464098],
 [9.0, 0.3628183949734276, 0.4408861857435958, 0.13129837213006507],
 [8.0, 0.3623251507985085, 0.4394945123104102, 0.13513755799699895],
 [9.5, 0.36051810512867954, 0.43839204361662265, 0.13315908195648835],
 [8.5, 0.3597629987587144, 0.440094394295939, 0.13528863884583778],
 [7.0, 0.3574157910817314, 0.4357392284301309, 0.13575448732562678],
 [6.5, 0.3571480587524357, 0.43471564446898003, 0.13501414585936694],
 [7.5, 0.35633529300113514, 0.43441008626314975, 0.13633516850227909],
 [5.5, 0.35045527138076066, 0.4247491026570801, 0.13260608574209778],
 [6.0, 0.3503767726499682, 0.42360994858021006, 0.1333123736889443],
 [5.0, 0.3486040402767273, 0.4253135064499454, 0.1337106279009972],
 [4.0, 0.34435912502347854, 0.41643962807336105, 0.1324311198498365],
 [4.5, 0.3424705960086811, 0.41638542665409883, 0.1315793889648043],
 [3.5, 0.3370436435125208, 0.40952792182902686, 0.13061871362556607],
 [3.0, 0.32788232376722665,

In [103]:
for cata in getListGames(split="challenge"):
    
    file=os.path.join(args.source_challenge_path,cata,'results_spotting.json')
    with open(file,'r') as f:
        data=json.load(f)
    pres=data['predictions']
    pres.sort(key=lambda x:(int(x['half']),int(x['position'])))
    
    json_data = {} 
    total_pre={}
    json_data["UrlLocal"]=data['UrlLocal']
    json_data["predictions"]=[]
    
    last_half=1
    start,end=0,window_size
    window=[]
    for cur_pre,pre in enumerate(pres):

        
        if start<= int(pre['position'])/1000.0 <end:
            window.append(pre)
        else:
            json_data["predictions"]+=process_window(window)
            window=[pre]
            start+=window_size
            end+=window_size
            
            if last_half==1 and int(pre['half'])==2:
                last_half=2
                start,end=0,window_size
         
    json_data["predictions"]+=process_window(window)
    
    os.makedirs(os.path.join(args.output_challenge_path,cata),exist_ok='True')
    with open(os.path.join(args.output_challenge_path,cata,'results_spotting.json'),'w') as f:
        json.dump(json_data,f,indent=2)
    
    
